In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
# Load the datasets
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

# Merge Transactions with Customers and Products
merged_data = transactions.merge(customers, on="CustomerID", how="left")
merged_data = merged_data.merge(products, on="ProductID", how="left")

# Display the merged data
print(merged_data.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region  SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36   300.68  Travis Campbell  South America  2024-04-11   
4      902.04   300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  Price_y  
0  ComfortLiving Bluetooth Speaker  Electronics   300.68  
1  ComfortLiving Bluetooth Speaker

In [6]:
# Initialize LabelEncoder
le = LabelEncoder()

# Encode categorical columns
merged_data["Region"] = le.fit_transform(merged_data["Region"])
merged_data["Category"] = le.fit_transform(merged_data["Category"])
merged_data["ProductName"] = le.fit_transform(merged_data["ProductName"])


In [10]:
print(merged_data.columns)


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y'],
      dtype='object')


In [11]:
products = pd.read_csv("Products.csv")
print(products.columns)


Index(['ProductID', 'ProductName', 'Category', 'Price'], dtype='object')


In [12]:
merged_data = transactions.merge(customers, on="CustomerID", how="left")
merged_data = merged_data.merge(products, on="ProductID", how="left")
print(merged_data.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region  SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36   300.68  Travis Campbell  South America  2024-04-11   
4      902.04   300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  Price_y  
0  ComfortLiving Bluetooth Speaker  Electronics   300.68  
1  ComfortLiving Bluetooth Speaker

In [14]:
# Use Price_y (from Products.csv) in aggregation
customer_data = merged_data.groupby("CustomerID").agg({
    "TotalValue": "sum",     # Total spending
    "Quantity": "sum",       # Total quantity purchased
    "Price_y": "mean",       # Average price of products purchased
    "Region": "first",       # Region
    "Category": "nunique"    # Number of unique product categories purchased
}).reset_index()

print(customer_data.head())


  CustomerID  TotalValue  Quantity     Price_y         Region  Category
0      C0001     3354.52        12  278.334000  South America         3
1      C0002     1862.74        10  208.920000           Asia         2
2      C0003     2725.38        14  195.707500  South America         3
3      C0004     5354.88        23  240.636250  South America         3
4      C0005     2034.24         7  291.603333           Asia         2


In [15]:
# Rename the columns to clarify their sources
merged_data = merged_data.rename(columns={
    "Price_x": "TransactionPrice",
    "Price_y": "StandardPrice"
})

# Verify the updated column names
print(merged_data.columns)


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'TransactionPrice', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'StandardPrice'],
      dtype='object')


In [16]:
# Rename the Price columns for clarity
merged_data = merged_data.rename(columns={
    "Price_x": "TransactionPrice",
    "Price_y": "StandardPrice"
})

# Aggregate customer-level data
customer_data = merged_data.groupby("CustomerID").agg({
    "TotalValue": "sum",       # Total spending
    "Quantity": "sum",         # Total quantity purchased
    "StandardPrice": "mean",   # Average standard price of products purchased
    "Region": "first",         # Region (as categorical)
    "Category": "nunique"      # Number of unique product categories purchased
}).reset_index()

# View the aggregated customer data
print(customer_data.head())


  CustomerID  TotalValue  Quantity  StandardPrice         Region  Category
0      C0001     3354.52        12     278.334000  South America         3
1      C0002     1862.74        10     208.920000           Asia         2
2      C0003     2725.38        14     195.707500  South America         3
3      C0004     5354.88        23     240.636250  South America         3
4      C0005     2034.24         7     291.603333           Asia         2


In [17]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode categorical variables
label_encoder = LabelEncoder()
customer_data["RegionEncoded"] = label_encoder.fit_transform(customer_data["Region"])

# Select features for similarity calculation
features = ["TotalValue", "Quantity", "StandardPrice", "Category", "RegionEncoded"]

# Normalize features
scaler = StandardScaler()
normalized_data = scaler.fit_transform(customer_data[features])

# Convert normalized data back into a DataFrame for easier handling
normalized_df = pd.DataFrame(normalized_data, columns=features, index=customer_data["CustomerID"])


In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(normalized_df)

# Create a DataFrame for similarity scores
similarity_df = pd.DataFrame(similarity_matrix, index=customer_data["CustomerID"], columns=customer_data["CustomerID"])

# Function to get top 3 similar customers
def get_top_lookalikes(similarity_df, customer_id, top_n=3):
    # Get similarity scores for the given customer
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:top_n + 1]
    return list(zip(similar_customers.index, similar_customers.values))

# Generate Lookalike.csv for the first 20 customers
lookalikes = {}
for customer_id in customer_data["CustomerID"][:20]:
    lookalikes[customer_id] = get_top_lookalikes(similarity_df, customer_id)

# Convert to a DataFrame for saving
lookalike_df = pd.DataFrame({
    "CustomerID": lookalikes.keys(),
    "Lookalikes": [str(v) for v in lookalikes.values()]
})

# Save Lookalike.csv
lookalike_df.to_csv("Lookalike.csv", index=False)
print("Lookalike.csv generated successfully!")


Lookalike.csv generated successfully!


In [21]:
# Load and display the Lookalike.csv
lookalike_output = pd.read_csv("Lookalike.csv")
print(lookalike_output.head())


  CustomerID                                         Lookalikes
0      C0001  [('C0011', 0.9662034972155572), ('C0152', 0.96...
1      C0002  [('C0142', 0.9868685460668792), ('C0159', 0.95...
2      C0003  [('C0174', 0.9422833940136143), ('C0047', 0.89...
3      C0004  [('C0165', 0.9770876758426572), ('C0018', 0.95...
4      C0005  [('C0140', 0.9915541185777954), ('C0186', 0.98...
